# Notebook 1: Recolección de Datos

## Indice
1. Introducción y Contexto
2. Configuracion del entorno
3. Carga de Raw Data
4. Extraccion de Metadata
5. Consolidacion del Dataset
6. Estadisticas Iniciales
7. Guardado de Datos Procesados


# Recolección de Datos del Plebiscito 2020

## Contexto
Este notebook documenta el proceso de recolección y extracción de metadatos del dataset de Harvard Dataverse sobre el Plebiscito Constitucional de Chile 2020

## Objetivos
1. Cargar IDs de tweets de los 3 archivos del dataset
2. Extraer información temporal de cada ID (Snowflake)
3. Crear dataset consolidado con metadatos
4. Generar estadísticas preliminares

## Metodologia
En primero momento intentamos hacer la hidratación tradicional de los datos a partir la API de X(Twitter), pero la versión gratuita solo permite hacerlo a 100 posts, y el hecho que haya que pagar para acceder a la API dificulta el acceso, por ende bajo este contexto, sumado con la probabilidad que muchos posts de esa época pueden haber sido eliminados y no sean accesibles de ver, decidimos hacer un análisis a traves de la extracción directa de timestamps desde los Snowflake IDs de X(Twitter)
Encontramos que este enfoque, hasta esta etapa del proyecto permite tanto la disponibillidad de los datos como ayudar a responder nuestras preguntas de investigación

# Configuracion del Entorno

In [2]:
import pandas as pd
from datetime import datetime
import json

# Carga de datos crudos

**Dataset:** Plebiscito Constitucional Chile 2020  
**Repositorio:** Harvard Dataverse  
**DOI:** 10.7910/DVN/WGU9R9W  
**Total IDs:** 1,704,494 tweets  
**Período:** 14 Octubre - 26 Noviembre 2020

### Archivos:
1. Plebiscito_Constitucional_Chile_2020_01.txt (1.17M IDs)
2. Plebiscito_Constitucional_Chile_2020_02.txt (~500k IDs)
3. Plebiscito_Constitucional_Chile_2020_03.txt (~200k IDs)

In [14]:
# Configuracion de archivos
archivos_config = [
    {
        'path': '../data/raw/Plebiscito_Constitucional_Chile_2020_01.txt',
        'muestra_size': 500000,
        'nombre': 'Archivo 1'
    },
    {
        'path': '../data/raw/Plebiscito_Constitucional_Chile_2020_02.txt',
        'muestra_size': 200000,
        'nombre': 'Archivo 2'
    },
    {
        'path': '../data/raw/Plebiscito_Constitucional_Chile_2020_03.txt',
        'muestra_size': 100000,
        'nombre': 'Archivo 3'
    }
]
fecha_plebiscito = datetime(2020, 10, 25)

# Funciones Auxiliares 
def extraer_timestamp_de_id(tweet_id):
    """
    Esta función extrae el timestamp del Sonwflake ID de X (Twitter)
    Los IDs de Twitter contienen información temporal embebida:
    - 41 bits: timestamp (ms desde Twitter epoch) --> "Twitter epoch" se refiere a la fecha de inicio utilizada por el sistema de IDs de Twitter (ahora X), que es el 4 de noviembre de 2010, a las 01:42:54.657 UTC
    - 10 bits: machine ID
    - 12 bits: sequence number

    Args:
        tweet_id (str/int): ID del Tweet

    Returns:
        datetime: Fecha y hora del tweet/ None si hay error
    """
    try:
        twitter_epoch = 1288834974657
        tweet_id_int = int(tweet_id)
        timestamp_ms = (tweet_id_int >> 22) + twitter_epoch
        return datetime.fromtimestamp(timestamp_ms / 1000)
    except:
        return None
def obtener_muestra_estratificada(tweet_ids, muestra_size):
    """
    Esta función obtiene una muestra distribuida uniformemente

    Implementa mustreo sistematico: toma 1 tweet cada k tweets para mantener representación
    temporal proporcional.

    Args:
        tweet_ids (list): Lista de IDs de tweets
        muestra_size (int): Tamaño deseado de muestra

    Returns:
        list: Muestra estratificada de IDs

    Ejemplo de lo que nos referimos a estratificada:
        >>> ids = ['1', '2', '3', '4', '5', '6']
        >>> muestra = obtener_muestra_estratificada(ids, 3)
        >>> print(muestra)
        ['1', '3', '5']
    """
    if len(tweet_ids) <= muestra_size:
        return tweet_ids
    
    # Calcular paso para distribución uniforme
    paso = len(tweet_ids) // muestra_size
    
    # Tomar tweets distribuidos uniformemente
    muestra = []
    for i in range(0, len(tweet_ids), paso):
        if len(muestra) >= muestra_size:
            break
        muestra.append(tweet_ids[i])
    
    return muestra

# Clasificar ventanas temporales
def clasificar_ventana(dias):
    if dias > 30:
        return '4. Más de 30 días antes'
    elif dias > 7:
        return '3. Entre 8-30 días antes'
    elif dias > 0:
        return '2. Última semana (1-7 días)'
    elif dias == 0:
        return '1. Día del plebiscito'
    else:
        return '0. Post-plebiscito'

# Período del día
def clasificar_periodo_dia(hora):
    if 5 <= hora < 12:
        return 'Mañana (5-12h)'
    elif 12 <= hora < 18:
        return 'Tarde (12-18h)'
    elif 18 <= hora < 23:
        return 'Noche (18-23h)'
    else:
        return 'Madrugada (23-5h)'

In [12]:
# Procesar los archivos de IDs de tweets
todos_los_datos = [] #Lista vacía donde se guardarán todos los tweets procesados de los 3 archivos
estadisticas_archivos = [] #Lista para guardar un resumen estadístico de cada archivo

for idx, config in enumerate(archivos_config, 1):
    # voy a recorrer cada elemento de archivos_config (que contiene 3 diccionarios con la info de los 3 archivos que quiero procesar)
    archivo = config['path']
    muestra_size = config['muestra_size']
    nombre = config['nombre']

    print(f"\n{'='*70}")
    print(f"PROCESANDO {nombre.upper()} ({idx}/3)")
    print(f"{'='*70}")

    print(f"\n Cargando IDs de {nombre}")

    tweet_ids = [] #Lista donde se guardaran los IDs del archivo actual
    with open(archivo, 'r', encoding='utf-8') as file:
        for linea in file:
            tweet_id = linea.strip()
            if tweet_id and tweet_id.isdigit():
                tweet_ids.append(tweet_id)
    total_ids = len(tweet_ids)
    print(f"-> {total_ids:,} IDs totales en el archivo")

    # Obtener muestra estratificada
    print(f"\n Tomando muestra estratificada...")
    print(f"   Objetivo: {muestra_size:,} tweets ({muestra_size/total_ids*100:.1f}% del archivo)") #Muestro que porcentaje representa la muestra del total
    
    muestra = obtener_muestra_estratificada(tweet_ids, muestra_size)
    print(f" Muestra obtenida: {len(muestra):,} tweets")
    print(f"   Distribución: 1 tweet cada {total_ids//len(muestra)} tweets")

    print(f"\n Extrayendo metadatos de la muestra...")
    datos_archivo = [] #Lista para guardar los datos procesados de ESTE archivo
    errores = 0 #contador para ids que no se pudieron procesar

    for i, tweet_id in enumerate(muestra):
        #recorro cada ID de la muestra y llamo a mi función auxiliar
        timestamp = extraer_timestamp_de_id(tweet_id)
        if timestamp:
            datos_archivo.append({
                'tweet_id': tweet_id, #el id original
                'timestamp': timestamp, #fecha y hora completa
                'fecha': timestamp.date(), #solo la fecha (sin hora)
                'hora': timestamp.hour, #solo el numero de la hora 0-23
                'dia_semana': timestamp.weekday(), #numero del dia (0=Lunes, 6=Domingo)
                'nombre_dia': timestamp.strftime('%A'), #nombre del dia en ingles
                'archivo_origen': nombre #de que archivo viene 
            })
        else:
            errores +=1

    # cada 20k tweets procesados, muestro el progreso
        if (i + 1) % 20000 == 0:
            print(f"  Progreso: {i+1:,}/{len(muestra):,} ({(i+1)/len(muestra)*100:.1f}%)")
    
    print(f"\n Procesamiento completado")
    print(f"  - Tweets válidos: {len(datos_archivo):,}")
    print(f"  - Errores: {errores:,}")
    print(f"  - Tasa de éxito: {len(datos_archivo)/len(muestra)*100:.2f}%")

    # Guardar estadísticas del archivo
    if datos_archivo:
        df_temp = pd.DataFrame(datos_archivo) # Creo un df temporal con los datos
        # guardo las estadisticas del archivo en un diccionario
        estadisticas_archivos.append({
            'archivo': nombre,
            'ids_totales': total_ids,
            'muestra_procesada': len(datos_archivo),
            'porcentaje_muestra': f"{len(datos_archivo)/total_ids*100:.2f}%",
            'fecha_inicio': df_temp['fecha'].min(),
            'fecha_fin': df_temp['fecha'].max(),
            'dias_cobertura': (df_temp['fecha'].max() - df_temp['fecha'].min()).days
        })

        # Muestro el rango de fechas y cuantos dias cubren los datos recolectados
        print(f"  - Período: {df_temp['fecha'].min()} a {df_temp['fecha'].max()}")
        print(f"  - Cobertura temporal: {(df_temp['fecha'].max() - df_temp['fecha'].min()).days} días")
    
    # Agregar todos los datos de este archivo a la lista general, sin que sea un nuevo elemento en la lista
    todos_los_datos.extend(datos_archivo)
    # Muestro cuantos tweets se han procesado en total hasta ahora (suma de todos los archivos)
    print(f"\n Total acumulado: {len(todos_los_datos):,} tweets")
            


PROCESANDO ARCHIVO 1 (1/3)

 Cargando IDs de Archivo 1
-> 1,174,133 IDs totales en el archivo

 Tomando muestra estratificada...
   Objetivo: 500,000 tweets (42.6% del archivo)
 Muestra obtenida: 500,000 tweets
   Distribución: 1 tweet cada 2 tweets

 Extrayendo metadatos de la muestra...
  Progreso: 20,000/500,000 (4.0%)
  Progreso: 40,000/500,000 (8.0%)
  Progreso: 60,000/500,000 (12.0%)
  Progreso: 80,000/500,000 (16.0%)
  Progreso: 100,000/500,000 (20.0%)
  Progreso: 120,000/500,000 (24.0%)
  Progreso: 140,000/500,000 (28.0%)
  Progreso: 160,000/500,000 (32.0%)
  Progreso: 180,000/500,000 (36.0%)
  Progreso: 200,000/500,000 (40.0%)
  Progreso: 220,000/500,000 (44.0%)
  Progreso: 240,000/500,000 (48.0%)
  Progreso: 260,000/500,000 (52.0%)
  Progreso: 280,000/500,000 (56.0%)
  Progreso: 300,000/500,000 (60.0%)
  Progreso: 320,000/500,000 (64.0%)
  Progreso: 340,000/500,000 (68.0%)
  Progreso: 360,000/500,000 (72.0%)
  Progreso: 380,000/500,000 (76.0%)
  Progreso: 400,000/500,000 (80

In [16]:
print(f"\n{'='*70}")
print("CONSOLIDANDO DATASET COMPLETO")
print(f"{'='*70}")

if not todos_los_datos:
    print("No se procesaron datos. Verifica las rutas de los archivos.")
    exit()

df_completo = pd.DataFrame(todos_los_datos)

print(f"\n Dataset consolidado: {len(df_completo):,} tweets")
print(f"  Desde: {df_completo['fecha'].min()}")
print(f"  Hasta: {df_completo['fecha'].max()}")

# Calcular días antes del plebiscito
df_completo['dias_antes_plebiscito'] = (fecha_plebiscito - pd.to_datetime(df_completo['fecha'])).dt.days

df_completo['ventana_temporal'] = df_completo['dias_antes_plebiscito'].apply(clasificar_ventana)

df_completo['periodo_dia'] = df_completo['hora'].apply(clasificar_periodo_dia)


CONSOLIDANDO DATASET COMPLETO

 Dataset consolidado: 800,000 tweets
  Desde: 2020-10-07
  Hasta: 2020-11-03


In [17]:
print(f"\n{'='*70}")
print(" ESTADÍSTICAS DEL DATASET ")
print(f"{'='*70}")

print(f"\n  RESUMEN GENERAL:")
print(f"   Total tweets en muestra:  {len(df_completo):,}")
print(f"   Período completo:         {df_completo['fecha'].min()} a {df_completo['fecha'].max()}")
print(f"   Días de cobertura:        {(df_completo['fecha'].max() - df_completo['fecha'].min()).days}")
print(f"   Variables por tweet:      {len(df_completo.columns)}")

print(f"\n*  DESGLOSE POR ARCHIVO:")
df_stats = pd.DataFrame(estadisticas_archivos)

print(f"\n*  DISTRIBUCIÓN POR VENTANA TEMPORAL:")
ventanas = df_completo['ventana_temporal'].value_counts().sort_index()
for ventana, count in ventanas.items():
    print(f"   {ventana}: {count:,} tweets ({count/len(df_completo)*100:.2f}%)")

print(f"\n*  TOP 10 DÍAS CON MÁS ACTIVIDAD:")
top_dias = df_completo['fecha'].value_counts().head(10)

for i, (fecha, count) in enumerate(top_dias.items(), 1):
    dias_antes = (fecha_plebiscito.date() - fecha).days
    print(f"   {i:2d}. {fecha} ({dias_antes:3d} días antes): {count:6,} tweets")

print(f"\n*  ACTIVIDAD POR HORA DEL DÍA:")
hora_counts = df_completo['hora'].value_counts().sort_index()
print(f"   Hora más activa:   {hora_counts.idxmax()}:00 hrs ({hora_counts.max():,} tweets)")
print(f"   Hora menos activa: {hora_counts.idxmin()}:00 hrs ({hora_counts.min():,} tweets)")

print(f"\n*  DISTRIBUCIÓN POR PERÍODO DEL DÍA:")
periodo_counts = df_completo['periodo_dia'].value_counts()
for periodo in ['Madrugada (23-5h)', 'Mañana (5-12h)', 'Tarde (12-18h)', 'Noche (18-23h)']:
    if periodo in periodo_counts.index:
        count = periodo_counts[periodo]
        print(f"   {periodo:20s}: {count:8,} tweets ({count/len(df_completo)*100:.2f}%)")




 ESTADÍSTICAS DEL DATASET 

  RESUMEN GENERAL:
   Total tweets en muestra:  800,000
   Período completo:         2020-10-07 a 2020-11-03
   Días de cobertura:        27
   Variables por tweet:      10

*  DESGLOSE POR ARCHIVO:

*  DISTRIBUCIÓN POR VENTANA TEMPORAL:
   0. Post-plebiscito: 70,345 tweets (8.79%)
   1. Día del plebiscito: 151,685 tweets (18.96%)
   2. Última semana (1-7 días): 348,989 tweets (43.62%)
   3. Entre 8-30 días antes: 228,981 tweets (28.62%)

*  TOP 10 DÍAS CON MÁS ACTIVIDAD:
    1. 2020-10-25 (  0 días antes): 151,685 tweets
    2. 2020-10-24 (  1 días antes): 66,310 tweets
    3. 2020-10-18 (  7 días antes): 63,945 tweets
    4. 2020-10-22 (  3 días antes): 46,244 tweets
    5. 2020-10-19 (  6 días antes): 45,123 tweets
    6. 2020-10-21 (  4 días antes): 44,800 tweets
    7. 2020-10-20 (  5 días antes): 44,440 tweets
    8. 2020-10-26 ( -1 días antes): 41,355 tweets
    9. 2020-10-10 ( 15 días antes): 39,005 tweets
   10. 2020-10-23 (  2 días antes): 38,127 

In [20]:
print(f"\n{'='*70}")
print(" GUARDANDO DATOS")
print(f"{'='*70}")

# 1. Guardo el DataFrame principal con todos los tweets
if todos_los_datos:
    df_completo = pd.DataFrame(todos_los_datos)
    
    # Guardar en CSV
    ruta_csv = '../data/processed/muestra_tweets.csv'
    df_completo.to_csv(ruta_csv, index=False, encoding='utf-8')
    print(f"\n DataFrame guardado en CSV:")
    print(f"  {ruta_csv}")

    # Guardar en Parquet (más eficiente para cargar después)
    ruta_parquet = '../data/processed/muestra_tweets.parquet'
    df_completo.to_parquet(ruta_parquet, index=False)
    print(f"\n DataFrame guardado en Parquet:")
    print(f"  {ruta_parquet}")

# 2. Guardar estadísticas de los archivos
if estadisticas_archivos:
    df_stats = pd.DataFrame(estadisticas_archivos)
    ruta_stats = '../data/processed/estadisticas_archivos.csv'
    df_stats.to_csv(ruta_stats, index=False, encoding='utf-8')
    print(f"\n Estadísticas guardadas:")
    print(f"  {ruta_stats}")



 GUARDANDO DATOS

 DataFrame guardado en CSV:
  ../data/processed/muestra_tweets.csv

 DataFrame guardado en Parquet:
  ../data/processed/muestra_tweets.parquet

 Estadísticas guardadas:
  ../data/processed/estadisticas_archivos.csv
